# The learning algorithm

The learning algorithm I implemented is based on an [implementation](https://github.com/fjonck/deep-reinforcement-learning/tree/master/ddpg-pendulum) provided by Udacity, which uses a DDPG approach to solve the [OpenAI Gym's Pendulum environment](https://github.com/openai/gym/wiki/Pendulum-v0). I made major changes to the algorithm to be able to solve the [Unity ML-Agents Tennis Environment](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md#tennis). It is based on the paper ["Continuous control with deep reinforcement learning"](https://arxiv.org/abs/1509.02971)[^1] with some [implementation details](https://spinningup.openai.com/en/latest/algorithms/ddpg.html) from "Spinning Up in Deep RL!" by OpenAI, an artificial intelligence research laboratory. The tennis-environment has actually two agents (an obvious fact in tennis) playing, each with their own observation space. But since both agents are supposed to learn exactly the same behaviour, which is keeping the ball in play, we use shared actor and critic networks. This means, we only have one instance of an agent. Each timestep, each agent stores its experience in a shared replay buffer. We could have solved the problem using an [Multi-Agent-DDPG](https://arxiv.org/abs/1706.02275)[^8]-approach. But this seemed to be a bit of an overkill, since the tennis agents neither need to learn differenct behaviour nor need to communicate. 

### The implementation details

The implementation at hand is a *actor-critic* and model-free algorithm with a deterministic policy as the actor and a Q-function as the critic, both approximized using *Deep Neural Networks*. It uses the actor function $\mu(s|\theta^\mu)$ to describe the policy by mapping a state to the best possible action to take. The action space is continuous. The critic function $Q^*(s,a)$ uses the state output $a = \mu(s|\theta^\mu)$ of the actor function to calculate the approximated expected reward. It is learned using the Bellman equation as in Q-learning using the squared L2 norm. For the actor, we just use gradient ascent on $Q^*(s,\mu(s|\theta^\mu))$ in order to maximize it. For both our optimization problems, we use the Adam optimizer[^2]. Additionally we use the features *Experience Replay* with a constant buffer size, *Fixed Targets* and *Soft Updates* of those targets. The following pseudo-code of the algorithm is a variation of the combination of the paper ["Continuous control with deep reinforcement learning"](https://arxiv.org/abs/1509.02971)[^1] and the [implementation](https://spinningup.openai.com/en/latest/algorithms/ddpg.html) from "Spinning Up in Deep RL!" by OpenAI. We use the notation of the [Udacity cheatsheet](https://github.com/udacity/deep-reinforcement-learning/blob/master/cheatsheet/cheatsheet.pdf):

**for** $i \leftarrow 1$ to *num_episodes* **do** 
> Observe $S_0$  
$t \leftarrow 0$  
**repeat**  
>>Choose action $A_t = \textrm{clip}(\mu(S_t| \theta^\mu) + \epsilon, -1, 1)$, where $\epsilon \sim \mathcal{N}(0,0.2)$
Take action $A_t$ and observe $R_{t+1} , S_{t+1}$  
Store experience $(S_t, A_t, R_t, R_{t+1})$ in $D$  
**Every** C steps **do**  
>>>Sample a random minibatch of experience tuples $(S_t, A_t, R_t, R_{t+1})$ from $D$  
Set the target $Y_t = R_{t+1} + \gamma \tilde{Q}(S_t,\tilde{\mu}(s|\theta^{\tilde{\mu}})|\theta^{\tilde{Q}})$ for the critic $Q$  
Update the critic $Q$ by one step of gradient descent according to the squared L2-Norm loss and the Adam optimizer  
Update the actor $\mu$ by one step of gradient ascent according to the Adam optimizer  
Perform soft updates $\theta^{\tilde{Q}} \leftarrow \tau\theta^Q + (1 - \tau)\theta^{\tilde{Q}}$ and $\theta^{\tilde{\mu}} \leftarrow \tau\theta^\mu + (1 - \tau)\theta^{\tilde{\mu}}$  

>>**end**  
$t \leftarrow t+1$  

>**until** $S_t$ is terminal;  

**end**  
**return** $Q$   

We use *Experience Replay* to prevent correlations of successive experiences. We can make use of rare occurrences of experiences and learn from them multiple times.  At every time step $t$ we store the experience tuple $(s_j, a_j, r_j, s_{j+1})$ in the replay buffer $D$ and sample from it a random minibatch to update the network. 
We use *Fixed Q-Targets* to prevent correlations which come the fact that we try to update the approximated action-value function $Q$ with an approximated action-value function $Q$ and an approximated policy $\mu$. To make use of the feature of fixed Q-targets, we introduce a target action-value function $\tilde{Q}$, a target policy $\tilde{\mu}$ and use them to compute the target $Y_t = R_{t+1} + \gamma \tilde{Q}(S_t,\tilde{\mu}(s|\theta^{\tilde{\mu}})|\theta^{\tilde{Q}})$. Every C steps, we soft update the parameters of the target action-value function $\tilde{Q}$ and the target policy $\tilde{\mu}$ by setting $\theta^{\tilde{Q}} \leftarrow \tau\theta^Q + (1 - \tau)\theta^{\tilde{Q}}$ and $\theta^{\tilde{\mu}} \leftarrow \tau\theta^\mu + (1 - \tau)\theta^{\tilde{\mu}}$. 
In the training step, right before the optimizer step, I clipped the gradient of the critic network to 1 to prevent exploding gradients. Additionally, the algorithm updates the target networks not at every time step, but only every 4th time an agent acts. As there are two agents, this is every 2nd timestep. This improved the performance of the learning algorithm a lot. 
Last but not least I introduced a Batch Normalization [Batch Normalization](https://arxiv.org/abs/1502.03167)[^3]in the actor model after the first and the second model and in the critic model after the first layer. This improved the performance of the training algorithm significantly. 

### The hyperparameters

	BUFFER_SIZE = int(1e6)  # replay buffer size
	BATCH_SIZE = 512        # minibatch size
	GAMMA = 0.99            # discount factor
	TAU = 1e-1              # for soft update of target parameters
	LR_ACTOR = 1e-4         # learning rate of the actor
	LR_CRITIC = 1e-3        # learning rate of the critic
	WEIGHT_DECAY = 0        # L2 weight decay
	UPDATE_EVERY = 4        # how often to update the network
	UPDATE_STEPS = 1        # how many times to update the network in a row

In difference to the paper ["Continuous control with deep reinforcement learning"](https://arxiv.org/abs/1509.02971)[^1] we changed the batch size to 512, the soft update factor $\tau$ to 0.1, the learning rate of the critic to 0.0001, disabled the weight decay and introduced the hyperparameters `UPDATE_EVERY = 4` and `UPDATE_STEPS = 1`. 
	

### The model architecture of the deep neural networks

#### The actor policy $\mu$

We use a 3-layer neural network which takes the 24-dimensional state as the input and deterministically outputs the predicted best possible action to take from that state. All the layers are fully connected. The first layer has 512 nodes and uses a ReLU-activation function after a Batch Normalization[^1]. The second layer has 256 nodes and uses a ReLU-activation function after a Batch Normalization[^1] as well. The third layer has 2 nodes which correspond to the entries of the action vector. We use a tanh-activation function to bound the action vector to the interval $[-1,1]$. 

#### The critic Q-function $Q$

We use a 3-layer neural network which takes the 24-dimensional state as the input to the first layer, the 2-dimensional action as the input to the second layer and outputs the predicted Q-value. All the layers are fully connected. The first layer has 512 nodes and uses a ReLU-activation function after a Batch Normalization[^1]. The second layer has 256 nodes and uses a ReLU-activation function after a Batch Normalization[^1] as well. The third layer has only one node which correspond to the discounted sum of future rewards. We don't use an activation function here as we want to predict the discounted sum of future rewards. Here is the implementation in PyTorch:

```python
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=512, fc2_units=256):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.bn1 = nn.BatchNorm1d(fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.bn2 = nn.BatchNorm1d(fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = F.relu(self.bn1(self.fc1(state)))
        x = F.relu(self.bn2(self.fc2(x)))
        return F.tanh(self.fc3(x))


class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, seed, fcs1_units=512, fc2_units=256):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fcs1 = nn.Linear(state_size, fcs1_units)
        self.bn1 = nn.BatchNorm1d(fcs1_units)
        self.fc2 = nn.Linear(fcs1_units+action_size, fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        self.reset_parameters()

    def reset_parameters(self):
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        xs = F.relu(self.bn1(self.fcs1(state)))
        x = torch.cat((xs, action), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)
```

### The performance of the learning algorithm

The algorithm at hand achieves an average reward of 0.5 over 100 consecutive episodes in 247 episodes. 

![](score.png) 

### Ideas for future work

The PPO-algorithm [PPO-algorithm](https://arxiv.org/abs/1707.06347)[^7]could be a better choice for solving the [Unity ML-Agents Tennis Environment](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md#tennis). It would be interesting to solve the problem using the [Multi-Agent-DDPG-algorithm](https://arxiv.org/abs/1706.02275)[^8]. In the plot you can see a high variance of the scores. This is probably due to the added noise to the action. To reduce the variance one could simply reduce the noise over the course of training. The performance could also be improved by using the following methods from the literature: Dueling DQN[^4], Prioritized Experience Replay[^5] or Double DQN[^3].


[^1]: Lillicrap, T. P., Hunt, J. J., Pritzel, A., Heess, N., Erez, T., Tassa, Y., Silver, D. & Wierstra, D. (2016). Continuous control with deep reinforcement learning.. In Y. Bengio & Y. LeCun (eds.), ICLR,. 

[^2]: Kingma, D. P. & Ba, J. (2014). Adam: A Method for Stochastic Optimization (cite arxiv:1412.6980 Comment: Published as a conference paper at the 3rd International Conference for Learning Representations, San Diego, 2015).

[^3]: Ioffe, S. & Szegedy, C. (2015). Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift.. _CoRR_, abs/1502.03167.

[^4]: Wang, Z., de Freitas, N. & Lanctot, M. (2015). Dueling Network Architectures for Deep Reinforcement Learning.. _CoRR_, abs/1511.06581.

[^5]: Schaul, T., Quan, J., Antonoglou, I. & Silver, D. (2015). Prioritized Experience Replay (cite arxiv:1511.05952Comment: Published at ICLR 2016).

[^6]: van Hasselt, H., Guez, A. & Silver, D. (2015). Deep Reinforcement Learning with Double Q-learning. arXiv:1509.06461v3.

[^7]: Schulman, J., Wolski, F., Dhariwal, P., Radford, A. & Klimov, O. (2017). Proximal Policy Optimization Algorithms.. _CoRR_, abs/1707.06347.

[^8]: Lowe, R., Wu, Y., Tamar, A., Harb, J., Abbeel, P. & Mordatch, I. (2017). Multi-Agent Actor-Critic for Mixed Cooperative-Competitive Environments.. In I. Guyon, U. von Luxburg, S. Bengio, H. M. Wallach, R. Fergus, S. V. N. Vishwanathan & R. Garnett (eds.), NIPS (p./pp. 6379-6390).